In [354]:
import librosa
import IPython
import music21 as mu
from midi2audio import FluidSynth
import pandas as pd

# update music21 environment so it can find MuseScore
mu.environment.set('musescoreDirectPNGPath', '/usr/bin/musescore')

# utility function for sound playback
def create_sound_file_from_midi(m21_data, file_name):
    m21_data.write('midi', fp = file_name + ".mid")
    fs = FluidSynth()
    fs.midi_to_audio('./' + file_name + ".mid", file_name + '.mp3')
    return(IPython.display.Audio("./" + file_name + ".mp3"))

In [378]:
# get hold of some data
mahler_ex = mu.converter.parse("./MidiAndMusicXMLFiles/MahlerSymphonies/symphony1/MusicXML/4th-movement-Sturmisch-bewegt.mxl")

In [379]:
# examine the list of parts


In [386]:
pList = []
partList = mahler_ex.getElementsByClass(mu.stream.Part)
for i in range(0, len(partList)):
    print(i, partList[i].getInstrument())
    pList.append(partList[i])

0 P1: Piccolo, Piccolo I II: Piccolo
1 P2: Flute, Fl�te I II III IV: Flute
2 P3: Oboe, Oboen I II III IV: Oboe
3 P4: B♭ Clarinet, Klarinetten in C I II III: B♭ Clarinet
4 P5: Alto Clarinet, Klarinetten in Es IV: Alto Clarinet
5 P6: Bassoon, Fagotte I II III: Bassoon
6 P7: Contrabassoon, Contrafagott: Contrabassoon
7 P8: Horn in F, H�rner in F I II: Horn in F
8 P9: Horn in F, H�rner in F III IV: Horn in F
9 P10: Horn in F, H�rner in F V VI: Horn in F
10 P11: Horn in F, H�rner in F VII: Horn in F
11 P12: B♭ Trumpet, Trompete in F V: B♭ Trumpet
12 P13: Trombone, Pausone IV: Trombone
13 P14: B♭ Trumpet, Trompeten in F I II: B♭ Trumpet
14 P15: B♭ Trumpet, Trompeten in F III IV: B♭ Trumpet
15 P16: Trombone, Pausonen I II: Trombone
16 P17: Trombone, Pausone III: Trombone
17 P18: Bass Trombone, Tuba: Bass Trombone
18 P19: Timpani, Pauken: Timpani
19 P20: Drumset, Triangel: Acoustic Bass Drum
20 P21: Drumset, Becken: Acoustic Bass Drum
21 P22: Drumset, Tam-Tam: Acoustic Bass Drum
22 P23: Timpan

In [381]:
# Look at contents

l = []

singlePart = pList[0]

for el in singlePart.flatten():

    l.append(type(el))

set(l)

{music21.bar.Barline,
 music21.clef.TrebleClef,
 music21.instrument.Piccolo,
 music21.key.KeySignature,
 music21.layout.SystemLayout,
 music21.meter.base.TimeSignature,
 music21.note.Note,
 music21.note.Rest,
 music21.tempo.MetronomeMark}

In [382]:
# Render a data frame 
events = []

currentNumerator = None
currentDenominator = None
currentInstrument = None
currentInstrumentName = None
currentPartName = None


for eachPart in pList:
    

    for el in eachPart.flatten():

        eventDictionary = {}
        eventDictionary['offset'] = el.offset
        eventDictionary['quarterLengthDuration'] = el.duration.quarterLength
        eventDictionary['measureNumber'] = el.measureNumber
        eventDictionary['currentNumerator'] = currentNumerator
        eventDictionary['currentDenominator'] = currentDenominator
        eventDictionary['instrument'] = currentInstrumentName
        eventDictionary['part'] = currentPartName


        currentType = str(type(el))

        if currentType == "<class 'music21.meter.base.TimeSignature'>":

            currentNumerator = el.numerator
            currentDenominator = el.denominator

        if "instrument" in currentType:

            currentInstrumentName = el.instrumentName
            currentPartName = el.partName


        if currentType == "<class 'music21.note.Note'>":
            eventDictionary['nameWithOctave'] = el.nameWithOctave
            eventDictionary['midiNumber'] = el.pitches[0].midi
            eventDictionary['fullName'] = el.pitches[0].fullName
            eventDictionary['name'] = el.pitches[0].name
            eventDictionary['octave'] = el.pitches[0].octave
            events.append(eventDictionary)

        elif currentType == "<class 'music21.chord.Chord'>":

            for eachNote in el:
                eventDictionary['nameWithOctave'] = eachNote.nameWithOctave
                eventDictionary['midiNumber'] = eachNote.pitches[0].midi
                eventDictionary['fullName'] = eachNote.pitches[0].fullName
                eventDictionary['name'] = eachNote.pitches[0].name
                eventDictionary['octave'] = eachNote.pitches[0].octave
                events.append(eventDictionary)
            

In [383]:
# examine different types of objects
x = pd.DataFrame(events)

In [384]:
x

,offset,quarterLengthDuration,measureNumber,currentNumerator,currentDenominator,instrument,part,nameWithOctave,midiNumber,fullName,name,octave
0,5.0,3.0,2,2,2,Piccolo,"Piccolo, Piccolo I II",D-5,73,D-flat in octave 5,D-,5
1,8.0,4.0,3,2,2,Piccolo,"Piccolo, Piccolo I II",D-5,73,D-flat in octave 5,D-,5
2,12.0,2.0,4,2,2,Piccolo,"Piccolo, Piccolo I II",D-5,73,D-flat in octave 5,D-,5
3,14.0,2.0,4,2,2,Piccolo,"Piccolo, Piccolo I II",C5,72,C in octave 5,C,5
4,16.0,4.0,5,2,2,Piccolo,"Piccolo, Piccolo I II",C5,72,C in octave 5,C,5
...,...,...,...,...,...,...,...,...,...,...,...,...
49662,698047/240,2.0,728,2,2,Contrabasses,"Contrabasses, Contrabass",D3,50,D-natural in octave 3,D,3
49663,699967/240,2.0,730,2,2,Contrabasses,"Contrabasses, Contrabass",A3,57,A-natural in octave 3,A,3
49664,699967/240,2.0,730,2,2,Contrabasses,"Contrabasses, Contrabass",A3,57,A-natural in octave 3,A,3
49665,701887/240,1.0,732,2,2,Contrabasses,"Contrabasses, Contrabass",D4,62,D-natural in octave 4,D,4


In [385]:
x.quarterLengthDuration.value_counts()

0.125      8564
0.5        7258
0.25       6378
2.0        5225
23/160     4694
1.0        3833
4.0        3474
137/480    2335
1/3        1824
2/3        1019
17/240      951
0.375       809
103/240     637
103/480     452
53/480      401
1.5         325
1/6         255
3.0         250
137/240     222
0.75        201
2/5         177
107/480     102
4/5          54
0.0625       52
17/480       31
17/160       29
4/3          18
1/12         17
0.09375      16
9/160        16
1/10         12
0.03125      10
6/5           8
137/160       6
1/5           4
0.1875        3
1/24          2
823/480       2
8/3           1
Name: quarterLengthDuration, dtype: int64